In [63]:
import numpy as np
import pandas as pd
import datetime as dt
from sklearn import preprocessing
from sklearn import linear_model

In [64]:
train_data = pd.read_csv('train.csv')

#columns converting
train_data['Functioning Day']=train_data['Functioning Day'].map(dict(Yes=1, No=0))

Holidays_map = {'Holiday': 1, 'No Holiday': 0}
train_data['Holiday']=train_data['Holiday'].map(Holidays_map)

seasons_map = {'Summer': 0, 'Autumn': 1,'Winter':2,'Spring':3}
train_data['Seasons']=train_data['Seasons'].map(seasons_map)

train_data['Date'] = pd.to_datetime(train_data['Date'])
train_data['Date']=train_data['Date'].map(dt.datetime.toordinal)

#df['y'] = df.drop(df[df['y'] < 0].index, inplace=True)
#df.drop( df[ df['y'] <= 0 ].index , inplace=True)

#rescaling data to make all data in range -1 ,1
#df= (df-df.mean()) / df.std()
train_data.columns=['ID','Date','y','Hour','temp','Humidity','Wind','Visibility','Dew','solar','Rainfall','Snowfall','Seasons','Holiday','Functioning Day']

train_data.head()

,ID,Date,y,Hour,temp,Humidity,Wind,Visibility,Dew,solar,Rainfall,Snowfall,Seasons,Holiday,Functioning Day
0,0,736341,254,0,-5.2,37,2.2,2000,-17.6,0.0,0.0,0.0,2,0,1
1,1,736341,204,1,-5.5,38,0.8,2000,-17.6,0.0,0.0,0.0,2,0,1
2,2,736341,173,2,-6.0,39,1.0,2000,-17.7,0.0,0.0,0.0,2,0,1
3,3,736341,107,3,-6.2,40,0.9,2000,-17.6,0.0,0.0,0.0,2,0,1
4,4,736341,78,4,-6.0,36,2.3,2000,-18.6,0.0,0.0,0.0,2,0,1


In [65]:
test_data = pd.read_csv('test.csv')
#columns converting
test_data['Functioning Day']=test_data['Functioning Day'].map(dict(Yes=1, No=0))

Holidays_map = {'Holiday': 1, 'No Holiday': 0}
test_data['Holiday']=test_data['Holiday'].map(Holidays_map)

seasons_map = {'Summer': 0, 'Autumn': 1,'Winter':2,'Spring':3}
test_data['Seasons']=test_data['Seasons'].map(seasons_map)

test_data['Date'] = pd.to_datetime(test_data['Date'])
test_data['Date']=test_data['Date'].map(dt.datetime.toordinal)
test_data.columns=['ID','Date','Hour','temp','Humidity','Wind','Visibility','Dew','solar','Rainfall','Snowfall','Seasons','Holiday','Functioning Day']

test_data.head()

,ID,Date,Hour,temp,Humidity,Wind,Visibility,Dew,solar,Rainfall,Snowfall,Seasons,Holiday,Functioning Day
0,5760,736684,0,-1.3,90,2.0,259,-2.7,0.0,0.0,2.2,2,0,1
1,5761,736684,1,-1.8,90,1.6,263,-3.2,0.0,0.0,2.2,2,0,1
2,5762,736684,2,-2.2,88,1.4,348,-3.9,0.0,0.0,2.2,2,0,1
3,5763,736684,3,-2.9,89,1.3,288,-4.4,0.0,0.0,2.2,2,0,1
4,5764,736684,4,-3.6,91,1.2,241,-4.8,0.0,0.0,2.2,2,0,1


In [66]:
#women = train_data.loc[train_data.Sex == 'female']["Survived"]
#rate_women = sum(women)/len(women)

#print("% of women who survived:", rate_women)

In [67]:
from sklearn.model_selection import train_test_split

train_data, val_df = train_test_split(train_data, test_size=0.4, random_state=42) # Try adding `stratify` here

X_train = train_data.drop(columns=['ID', 'y'])
y_train = train_data['y']

X_val = val_df.drop(columns=['ID', 'y'])
y_val = val_df['y']

In [68]:
from sklearn.linear_model import Lasso
from sklearn.model_selection import RepeatedKFold
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

X_test = test_data.drop(columns=['ID'])

# define model
model = Lasso(alpha=0.001)
# define model evaluation method
cv = RepeatedKFold(n_splits=10, n_repeats=10, random_state=1)
# evaluate model
scores = cross_val_score(model, X_train, y_train, scoring='neg_mean_absolute_error', cv=cv, n_jobs=-1)

regressor = LinearRegression()
regressor.fit(X_train, y_train)
y_train_predicted = regressor.predict(X_train)
y_test_predicted = regressor.predict(X_test)
test_data['y'] = y_test_predicted

from sklearn.metrics import r2_score
r2 = r2_score(y_train, y_train_predicted)
print('r2 score for perfect model is', r2)

test_data[['ID', 'y']].to_csv('submission.csv', index=False)
print("Your submission was successfully saved!")

test_data[['ID', 'y']].info()

r2 score for perfect model is 0.5458485366492958
Your submission was successfully saved!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000 entries, 0 to 2999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      3000 non-null   int64  
 1   y       3000 non-null   float64
dtypes: float64(1), int64(1)
memory usage: 47.0 KB


In [55]:
def rmsle(Y, y_pred):
    sum=0.0
    for x in range(len(y_pred)):
        if y_pred[x]<0 or Y[x]<0: #check for negative values
            continue
        p = np.log(y_pred[x]+1)
        r = np.log(Y[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(y_pred))**0.5
rmsle(y_train, y_train_predicted)
#print("RMSLE score for perfect model is : ",final_res[0])

KeyError: 8

In [9]:
#from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestClassifier

# Create an instance of the classifier
classifier = RandomForestClassifier(max_depth=25, random_state=0)

# Train the classifier
classifier = classifier.fit(X_train, y_train)


y_train_predicted = classifier.predict(X_train)

y_test_predicted = classifier.predict(X_test)
test_data['y'] = y_test_predicted




test_data[['ID', 'y']].to_csv('submission2.csv', index=False)
print("Your submission was successfully saved!")

Your submission was successfully saved!


In [ ]:
def rmsle(Y, y_pred):
    sum=0.0
    for x in range(len(y_pred)):
        if y_pred[x]<0 or Y[x]<0: #check for negative values
            continue
        p = np.log(y_pred[x]+1)
        r = np.log(Y[x]+1)
        sum = sum + (p - r)**2
    return (sum/len(y_pred))**0.5
rmsle(y_train, y_train_predicted)
#print("RMSLE score for perfect model is : ",final_res[0])